In [ ]:
# STEP 1: Install required libraries
!pip install --upgrade openai scikit-learn

# STEP 2: Imports
from openai import OpenAI
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

# STEP 3: API key (keep this private)
client = OpenAI(api_key="YOUR CHATGPT API KEY")

# STEP 4: Resume data and job description
user_data = """
Name: YOUR FULL NAME
Email: YOUR EMAIL
Phone: YOUR PHONE NUMBER
Location: YOUR LOCATION (optional)
LinkedIn: YOUR LINKEDIN URL
Website: YOUR PORTFOLIO URL

Skills:
- Python, JavaScript, TypeScript
- Angular, Ionic, Firebase, HTML, CSS, Tailwind
- GCP, GitHub Actions, Docker, REST APIs, CI/CD
- OpenAI API, Gemini API, Prompt Engineering
- Agile Methodology, Team Management

Experience:
YOUR EXPERIENCE at COMAPNY NAME (FROM – TILL)
- YOUR RESUME BULLET POINTS
- Led a team of 12+ and exceeded targets by 15%
- Improved operations using inventory systems and analytics

FOR EXAMPLE Freelance Software Developer (2023 – 2024)
- Built ChattyAI and Job4You using AI + Firebase + Angular

Projects:
YOUR PROJECTS OR ANYTHING ELSE YOU WANT TO ADD
FOR EXAMPLE Job4You  PROJECT URL (e.g. https://...)
- YOUR BULLET POINTS
- Resume generator and job automation platform

Education:
STUDYING/STUDIED, ORGANIZATION NAME (FROM – TILL)

Certifications:
- YOUR CERTIFICATES OR ACHIEVEMENTS
- AWS Certified Cloud Practitioner
- Google Cloud Digital Leader
"""

job_description = """
PASTE YOUR JOB DESCRIPTION HERE
"""

# STEP 5: ATS score functions
def calculate_ats_score(resume_text, job_description_text):
    texts = [resume_text.lower(), job_description_text.lower()]
    vect = CountVectorizer(stop_words='english').fit_transform(texts)
    return round(cosine_similarity(vect[0:1], vect[1:2])[0][0] * 100, 2)

def attach_ats_score_to_output(document_text, ats_score):
    return f"{document_text}\n\n---\n📊 ATS Score: {ats_score}%"

# STEP 6: GPT call
def generate_with_gpt(prompt, max_tokens=1500):
    resp = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role":"system","content":"You are a resume and cover letter expert."},
            {"role":"user","content":prompt}
        ],
        temperature=0.7,
        max_tokens=max_tokens
    )
    return resp.choices[0].message.content.strip()

# STEP 7: Extract job keywords
def extract_job_keywords(jd, top_n=25):
    vec = CountVectorizer(stop_words='english')
    mat = vec.fit_transform([jd])
    sums = np.asarray(mat.sum(axis=0)).flatten()
    freq = {w:sums[i] for w,i in vec.vocabulary_.items()}
    sorted_kws = sorted(freq.items(), key=lambda x:x[1], reverse=True)
    return [kw for kw,_ in sorted_kws[:top_n]]

extracted_keywords = extract_job_keywords(job_description, top_n=25)
formatted_keywords = ", ".join(extracted_keywords)

# STEP 8: Prompts
resume_prompt = f"""
Generate an ATS-optimized resume for {user_data} based on the following candidate profile and make it humanized and easy to read and understand using HARVARD RESUME STYLES and job description.

Requirements:
- Format: Plain text (no markdown), styled for Arial 11pt.
- Sections (in order):
  1. Contact Information
  2. Professional Summary (using userdata and job description.)
  3. Key Skills (grouped by relevant categories and if there is any skills from the job description show the  in the end of resume as a suggestions to add them.à)
  4. Work Experience (with concise bullet points and metrics)
  5. Projects (include your role, technologies used, and outcomes and using the link analyze the project and  ake it look good dfor the resume.)
  6. Education ( show all the achievements)
  7. Certifications
- Add missing keywords and skills which are not in user's resume  (for example; MISSING K&S: skill1, skill2, ...)from the job description at the bottom.
- ATS Optimization: Integrate keywords from the job description organically. ”.


USER:
{user_data}

JOB:
{job_description}
"""

cover_letter_prompt = f"""
Generate a professional, humanized with no ai detection and easy to read and understand , ATS-friendly cover letter for {user_data} applying to the role described below.

Requirements:
- Format: Plain text (no markdown), paragraph style.
- Sections (in order):
  1. Greeting: Address the hiring team or hiring manager by name if known.
  2. Introduction: Briefly state the position you’re applying for and why you’re excited.
  3. Body:
     - Alignment of Skills & Experience: Highlight 2–3 key qualifications from your profile that directly match the job requirements.
     - Value Proposition: Explain what you’ll bring to the team (e.g., leadership, technical impact).
  4. Closing & Call to Action: Express enthusiasm, reference attached resume, and invite next steps.
  5. Signature: “Sincerely,” plus your name.


USER:
{user_data}

JOB:
{job_description}
"""

# STEP 9: Strict resume optimizer
def gpt_strict_keyword_resume(user_data, jd, keywords):
    prompt = (
        "Rewrite the resume using the user's data. You MUST include these keywords exactly "
        "and integrate them naturally into the resume:\n\n"
        f"Keywords: {keywords}\n\n"
        "Use structure:\n"
        "1. Contact Info\n2. Summary\n3. Skills\n4. Experience\n5. Projects\n6. Education\n7. Certifications\n\n"
        "Optimize for 90%+ ATS.\n\n"
        f"USER INFO:\n{user_data}\n\nJOB DESCRIPTION:\n{jd}"
    )
    return generate_with_gpt(prompt)

# STEP 10: Generate documents & scores
resume = generate_with_gpt(resume_prompt)
resume_score = calculate_ats_score(resume, job_description)
resume_output = attach_ats_score_to_output(resume, resume_score)

cover_letter = generate_with_gpt(cover_letter_prompt, max_tokens=600)
cover_letter_score = calculate_ats_score(cover_letter, job_description)
cover_letter_output = attach_ats_score_to_output(cover_letter, cover_letter_score)

strict_resume = gpt_strict_keyword_resume(user_data, job_description, formatted_keywords)
strict_resume_score = calculate_ats_score(strict_resume, job_description)
strict_resume_output = attach_ats_score_to_output(strict_resume, strict_resume_score)

# STEP 11: Normalize resume text for matching
resume_norm = re.sub(r"[^\w\s]", " ", strict_resume.lower())
resume_tokens = set(resume_norm.split())

# STEP 12: Identify missing keywords
missing_keywords = [kw for kw in extracted_keywords if kw.lower() not in resume_tokens]

# STEP 13: Emails
# %%
import re

# 13a) Extract “Role” and “Company” from your job_description string
m = re.search(r'([A-Z][A-Za-z0-9 &\+\-]+?) at ([A-Z][A-Za-z0-9 &\+\-]+)', job_description)
if m:
    role    = m.group(1).strip()
    company = m.group(2).strip()
else:
    role    = "the position"
    company = "the company"

print(f"🔍 Detected Role: {role}  |  Company: {company}")

# 13b) Build & call GPT for the initial application email
app_prompt = f"""
Create a warm, professional initial email to the hiring team at {company} for the {role} role.

Requirements:
1. Greeting (e.g. “Dear Hiring Team,”).
2. One line stating the role and your excitement.
3. 2–3 sentences tying your top skills/experiences to key requirements in the job description.
4. Closing that references your attached resume and cover letter and invites next steps.
5. Sign-off (“Sincerely,” or “Best regards,”) + your name.

PROFILE:
{user_data}

JOB DESCRIPTION:
{job_description}
"""

application_email = generate_with_gpt(app_prompt, max_tokens=300)

# 13c) Build & call GPT for the follow-up email
followup_prompt = f"""
Write a concise, polite follow-up email regarding your application for the {role} role at {company}.

Requirements:
1. Subject line: “Follow-Up: {role} Application”.
2. Pleasant greeting.
3. One sentence reminding them of your application.
4. One sentence reaffirming your enthusiasm and fit.
5. Call to action (“I’d welcome the chance to discuss further.”).
6. Sign-off and your name.

PROFILE:
{user_data}

JOB DESCRIPTION:
{job_description}
"""

follow_up_email = generate_with_gpt(followup_prompt, max_tokens=300)

# STEP 14: Display outputs
print("📄 ORIGINAL RESUME:\n\n", resume_output)
print("\n📄 OPTIMIZED RESUME (ENFORCED KEYWORDS):\n\n", strict_resume_output)
print("\n📩 COVER LETTER:\n\n", cover_letter_output)
print("\n📧 APPLICATION EMAIL:\n\n", application_email)
print("\n🔁 FOLLOW-UP EMAIL:\n\n", follow_up_email)
print(f"\n🔍 ATS Score BEFORE: {resume_score}% | AFTER: {strict_resume_score}% ✅")

# STEP 15: Suggest missing skills
if strict_resume_score < 90:
    print("\n⚠️ To reach 90%+ ATS, consider adding these keywords:")
    print(", ".join(missing_keywords))
else:
    print("\n✅ Your resume meets the 90%+ ATS threshold!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.9/680.9 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.78.0
    Uninstalling openai-1.78.0:
      Successfully uninstalled openai-1.78.0


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: YOUR CHA******** KEY. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}